In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy
import cartopy.crs as ccrs
from matplotlib import colors
from scipy.integrate import trapezoid
from scipy.spatial import cKDTree
from matplotlib.colors import Normalize

In [25]:
pisco = pd.read_csv('contributor_folders/annie/PISCO_kelpforest_swath.1.11.csv', encoding='windows-1252')

/tmp/ipykernel_236/2558463444.py:1: DtypeWarning: Columns (12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  pisco = pd.read_csv('contributor_folders/annie/PISCO_kelpforest_swath.1.11.csv', encoding='windows-1252')


In [26]:
pisco.head()

,campus,method,survey_year,year,month,day,site,zone,transect,classcode,count,size,disease,observer,depth,notes,site_name_old
0,UCSC,SBTL_SWATH_PISCO,1999,1999,9,7,HOPKINS_DC,INNER,1,CRYSTE,1,NaN,NaN,STACEY BUCKELEW,6.1,NaN,NaN
1,UCSC,SBTL_SWATH_PISCO,1999,1999,9,7,HOPKINS_DC,INNER,1,DICSPP,2,NaN,NaN,STACEY BUCKELEW,6.1,NaN,NaN
2,UCSC,SBTL_SWATH_PISCO,1999,1999,9,7,HOPKINS_DC,INNER,1,MACPYRAD,2,2.0,NaN,STACEY BUCKELEW,6.1,NaN,NaN
3,UCSC,SBTL_SWATH_PISCO,1999,1999,9,7,HOPKINS_DC,INNER,1,MACPYRAD,1,5.0,NaN,STACEY BUCKELEW,6.1,NaN,NaN
4,UCSC,SBTL_SWATH_PISCO,1999,1999,9,7,HOPKINS_DC,INNER,1,MACPYRAD,1,7.0,NaN,STACEY BUCKELEW,6.1,NaN,NaN


In [27]:
pisco = pisco.drop(columns=['campus', 'survey_year', 'disease', 'observer', 'notes', 'site_name_old'])

In [28]:
pisco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285808 entries, 0 to 285807
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   method     285808 non-null  object 
 1   year       285808 non-null  int64  
 2   month      285808 non-null  int64  
 3   day        285808 non-null  int64  
 4   site       285808 non-null  object 
 5   zone       285808 non-null  object 
 6   transect   285808 non-null  int64  
 7   classcode  285808 non-null  object 
 8   count      285808 non-null  int64  
 9   size       112664 non-null  float64
 10  depth      265489 non-null  float64
dtypes: float64(2), int64(5), object(4)
memory usage: 24.0+ MB


In [29]:
piscoTaxon = pd.read_csv('contributor_folders/annie/PISCO_kelpforest_taxon_table.1.11.csv', encoding='windows-1252')
piscoTaxon.head()

,campus,sample_type,sample_subtype,classcode,Kingdom,Phylum,Class,Order,Family,Genus,...,LOOKED2015,LOOKED2016,LOOKED2017,LOOKED2018,LOOKED2019,LOOKED2020,LOOKED2021,LOOKED2022,LOOKED2023,LOOKED2024
0,UCSB,FISH,FISH,AARG,Animalia,Chordata,Actinopterygii,Perciformes,Embiotocidae,Amphistichus,...,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
1,UCSB,FISH,FISH,ACOR,Animalia,Chordata,Actinopterygii,Scorpaeniformes,Cottidae,Artedius,...,no,no,no,no,no,no,no,no,no,no
2,UCSC,FISH,FISH,ACOR,Animalia,Chordata,Actinopterygii,Scorpaeniformes,Cottidae,Artedius,...,no,no,no,no,no,no,no,no,no,no
3,UCSB,FISH,FISH,ADAV,Animalia,Chordata,Actinopterygii,Perciformes,Haemulidae,Anisotremus,...,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
4,UCSC,FISH,FISH,ADAV,Animalia,Chordata,Actinopterygii,Perciformes,Haemulidae,Anisotremus,...,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes


In [30]:
algaeInfo = piscoTaxon[(piscoTaxon['sample_type'] == 'SWATH') & (piscoTaxon['sample_subtype'] == 'ALGAE') & (piscoTaxon['LOOKED2024']=='yes')]

In [33]:
algaeInfo = algaeInfo.drop(columns=['campus', 'sample_type', 'sample_subtype', 'taxonomic_source', 'taxonomic_id', 'notes'])

In [37]:
kelpCodes = algaeInfo['classcode'].unique()

In [38]:
kelpCodes

array(['ALAMAR', 'COSCOS', 'EISARBAD', 'EISARBADSUB', 'EISARBJUV',
       'HEDSES', 'LAMFAR', 'LAMREC', 'LAMSETAD', 'LAMSETADSUB',
       'LAMSETJUV', 'LAMSTUMP', 'MACPYRAD', 'MACPYRJUV', 'MACPYRREC',
       'NEOFIM', 'NERLUE', 'NERLUEJUV', 'PELPOR', 'PLEGAR', 'PTECALAD',
       'PTECALADSUB', 'PTECALJUV', 'SACLAT', 'STEOSMAD', 'UNDPIN'],
      dtype=object)

In [39]:
piscoKelp = pisco[pisco['classcode'].isin(kelpCodes)]

In [40]:
piscoKelp

,method,year,month,day,site,zone,transect,classcode,count,size,depth
2,SBTL_SWATH_PISCO,1999,9,7,HOPKINS_DC,INNER,1,MACPYRAD,2,2.0,6.1
3,SBTL_SWATH_PISCO,1999,9,7,HOPKINS_DC,INNER,1,MACPYRAD,1,5.0,6.1
4,SBTL_SWATH_PISCO,1999,9,7,HOPKINS_DC,INNER,1,MACPYRAD,1,7.0,6.1
5,SBTL_SWATH_PISCO,1999,9,7,HOPKINS_DC,INNER,1,MACPYRAD,1,16.0,6.1
6,SBTL_SWATH_PISCO,1999,9,7,HOPKINS_DC,INNER,1,MACPYRAD,1,17.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...
285795,SBTL_SWATH_PISCO,2024,11,1,SCI_SCORPION_ANCHORAGE,OUTER,2,MACPYRAD,2,9.0,11.6
285796,SBTL_SWATH_PISCO,2024,11,1,SCI_SCORPION_ANCHORAGE,OUTER,2,MACPYRAD,1,10.0,11.3
285797,SBTL_SWATH_PISCO,2024,11,1,SCI_SCORPION_ANCHORAGE,OUTER,2,MACPYRAD,1,11.0,12.2
285798,SBTL_SWATH_PISCO,2024,11,1,SCI_SCORPION_ANCHORAGE,OUTER,2,MACPYRAD,1,15.0,12.2


In [41]:
piscoKelp.to_csv('contributor_folders/annie/PISCO_kelpdata.csv')

In [42]:
piscoSites = pd.read_csv('contributor_folders/annie/PISCO_kelpforest_site_table.1.11.csv', encoding='windows-1252')

In [43]:
piscoSites

,campus,method,survey_year,site,latitude,longitude,MPA_Name,mpa_type,site_status
0,UCSC,SBTL_FISH_PISCO,1999,HOPKINS_DC,36.623586,-121.904196,Lovers Point - Julia Platt SMR,SMR,mpa
1,UCSC,SBTL_FISH_PISCO,1999,HOPKINS_UC,36.621649,-121.900789,Lovers Point - Julia Platt SMR,SMR,mpa
2,UCSC,SBTL_FISH_PISCO,1999,MACABEE_UC,36.617148,-121.895695,Lovers Point - Julia Platt SMR,SMR,reference
3,UCSC,SBTL_FISH_PISCO,1999,MACABEE_DC,36.618184,-121.896835,Lovers Point - Julia Platt SMR,SMR,reference
4,UCSC,SBTL_FISH_PISCO,1999,STILLWATER_DC,36.559729,-121.944014,Carmel Bay SMCA,SMCA,mpa
...,...,...,...,...,...,...,...,...,...
10400,UCSB,SBTL_QUAD_PISCO,2006,COJO_E,34.441287,-120.416793,Point Conception SMR,SMR,reference
10401,UCSB,SBTL_QUAD_PISCO,2009,ANACAPA_EAST_FISH_CAMP_E,34.004500,-119.385783,Anacapa Island SMR,SMR,reference
10402,UCSB,SBTL_QUAD_PISCO,2009,ANACAPA_EAST_FISH_CAMP_E,34.004500,-119.385783,Anacapa Island SMR,SMR,reference
10403,UCSB,SBTL_QUAD_PISCO,2006,JALAMA_N,34.486960,-120.500523,Vandenberg SMR,SMR,reference


In [44]:
piscoSites = piscoSites[piscoSites['survey_year'] == 2024]

In [47]:
piscoSites = piscoSites[piscoSites['method'].str.contains('SWATH')]

In [51]:
piscoSites = piscoSites.drop(columns=['campus', 'survey_year'])

In [54]:
piscoSites.to_csv('data/PISCOSites.csv', index=False)